In [1]:
import os, sys, re, argparse, pdb, glob, pprint, json
import TS_pipe as tsp
import pandas as pd
f***REMOVED*** rdkit import Chem
f***REMOVED*** rdkit.Chem import rdmolfiles
f***REMOVED*** openbabel import pybel

json_file = '../results/greenfield_imines.json'
df = pd.read_json(json_file, orient='index')
df = df.reset_index(names='Molecule')

empty = df[df['TS guess key value'].isna()]
nonlin = df[df['TS guess key value'] < 160.0]

df = df[df['TS guess key value'] > 160.0]
df = df.reset_index()


df.count()

index                             13
Molecule                          13
TS mode                           13
key parameter atoms               13
TS guess key parameter            13
TS guess key value                13
pysis input conformers number     13
pysis output conformers           13
pysis output conformers number    13
pysis conformers properties       13
orca input conformers number      13
orca output conformers            13
orca output conformers number     13
orca conformers properties        13
dtype: int64

In [2]:
empty

,Molecule,TS mode,key parameter atoms,TS guess key parameter,TS guess key value,pysis input conformers number,pysis output conformers,pysis output conformers number,pysis conformers properties,orca input conformers number,orca output conformers,orca output conformers number,orca conformers properties
5,C11H11N3_1,Greenfield C-C=N-C,"[9, 1, 2, 3]",angle C=N-C,NaN,0,[],0,{},0,[],0,{}


In [3]:
#df.loc[81]['orca conformers properties']['C17H21N3/9_C17H21N3_orca/cregened_conformers_1_reopt']

In [4]:
failed_orca_count = 0
failed_orca_list = []
conformers_by_orca_count = df['pysis output conformers number'].sum()
conformers_to_orca_count = df['orca input conformers number'].sum()
energies_dict = {}
for i in df.index:
    Z_Es = []
    TS_Es = []
    for j, conf in enumerate(df.loc[i]['orca output conformers']):
        try:
            if not df.loc[i]['orca conformers properties'][conf]['orca TS conformer key value'] > 160.0:
                print(f'Small angle for conformer {conf}')
                continue
            if not list(df.loc[i]['orca conformers properties'][conf].values()):
                failed_orca_count += 1
                failed_orca_list.append(conf)
            if abs(df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value']) < 90.0:
                Z_energy = df.loc[i]['orca conformers properties'][conf]['orca reac1 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac1 E corr']
            else:
                Z_energy = df.loc[i]['orca conformers properties'][conf]['orca reac2 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac2 E corr']
            TS_energy =  df.loc[i]['orca conformers properties'][conf]['orca TS def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca TS E corr']
        except KeyError as e:
            print(f'Conformer {conf} is missing')
            print(e)
            continue
        Z_Es.append(Z_energy)
        TS_Es.append(TS_energy)
    if not ( Z_Es and TS_Es ):
        continue
    energies_dict[f"{df.loc[i]['Molecule']}"] = {}
    energies_dict[f"{df.loc[i]['Molecule']}"]['Z energies'] = Z_Es
    energies_dict[f"{df.loc[i]['Molecule']}"]['TS energies'] = TS_Es
print(f'Conformers generated by ORCA stage: {conformers_to_orca_count}')
print(f'Failed ORCA jobs: {failed_orca_count}')
print(f'Failed ORCA list:', failed_orca_list)
pprint.pprint(energies_dict)
print(len(energies_dict.keys()))

with open('greenfield_imines_final_energies.json', 'w') as json_file:
    json.dump(energies_dict, json_file, indent=4)



Conformer C12H13N3O_1/9_C12H13N3O_1_orca/cregened_conformers_3_reopt is missing
'orca TS conformer key value'
Conformers generated by ORCA stage: 46
Failed ORCA jobs: 0
Failed ORCA list: []
{'C11H10FN3': {'TS energies': [-689.571889454162, -689.571328832512],
               'Z energies': [-689.602156608522, -689.601013260509]},
 'C11H10FN3_1': {'TS energies': [-689.565208083824,
                                 -689.563963713598,
                                 -689.563475661549,
                                 -689.562455475633],
                 'Z energies': [-689.5932793210361,
                                -689.6018807531291,
                                -689.590856915409,
                                -689.592645154483]},
 'C11H10N4O2': {'TS energies': [-794.906572075104,
                                -794.906173485432,
                                -794.906149119878],
                'Z energies': [-794.931926707185,
                               -794.9313614455571

In [5]:
os.getcwd()

'C:\\Users\\***REMOVED***pe672\\Documents\\GitHub\\TS_pipeline\\src'